Initial coordinates + imports

In [ ]:
from ipyleaflet import Map, DrawControl, basemaps, basemap_to_tiles
import ipywidgets as widgets
from IPython.display import display

# Initial coordinates
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))

# Display the entered coordinates
print(f"Entered coordinates: {latitude, longitude}")


Interactive map + bounds var

In [19]:
# Create an interactive map centered on the specified coordinates
m = Map(
    basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),  # Satellite imagery basemap
    center=(latitude, longitude),  # Center of the map at the specified coordinates
    zoom=15  # Initial zoom level
)

# Variable to store the coordinates of the drawn shape
shape_coords = []

# Initialize bounds outside the function
bounds = []

# Function to handle the drawing event on the map
def handle_draw(target, action, geo_json):
    global shape_coords  # Use the global variable shape_coords
    if action == 'created':  # Check if a new shape has been created
        geometry_type = geo_json['geometry']['type']  # Get the geometry type
        
        if geometry_type == 'Polygon':  # Rectangle or Polygon
            coords = geo_json['geometry']['coordinates'][0]
            latitudes = [coord[1] for coord in coords]
            longitudes = [coord[0] for coord in coords]
            shape_coords = [
                (min(latitudes), min(longitudes)),  # SW
                (max(latitudes), max(longitudes))   # NE
            ]
            print(f"Rectangle selected: SW {shape_coords[0]}, NE {shape_coords[1]}")

        elif geometry_type == 'Point':  # Point
            coords = geo_json['geometry']['coordinates']
            shape_coords = [(coords[1], coords[0])]  # Store as a tuple (lat, lon)
            print(f"Point selected: {shape_coords[0]}")

# Create a drawing control with options for different shapes
draw_control = DrawControl(
    rectangle={'shapeOptions': {'color': 'blue'}},  # Style options for rectangles
    marker={},  # Option to draw a point
    edit=True  # Allow editing of drawn shapes
)

# Bind the handle_draw function to the drawing event
draw_control.on_draw(handle_draw)

# Add the drawing control to the map
m.add_control(draw_control)

# Function to update and display the coordinates of the selected shape
def update_coords(button):
    global bounds  # Use the global variable bounds
    if shape_coords:
        if len(shape_coords[0]) == 2:  # If it's a rectangle or a point
            if len(shape_coords) == 2:  # Rectangle (two points SW and NE)
                coord_label.value = f"Rectangle selected: SW {shape_coords[0]}, NE {shape_coords[1]}"
                bounds = [shape_coords[0][0], shape_coords[0][1], shape_coords[1][0], shape_coords[1][1]]
            else:  # Point (only one point)
                coord_label.value = f"Point selected: {shape_coords[0]}"
                bounds = [shape_coords[0][0], shape_coords[0][1]]
        print(f"Bounds updated: {bounds}")
    else:
        coord_label.value = "No shape selected."

# Create a Label widget to display the selected coordinates
coord_label = widgets.Label()

# Create a button that updates the selected coordinates when clicked
button = widgets.Button(description="Get Coordinates")
button.on_click(update_coords)  # Bind the update_coords function to the button click

# Display the interactive map, button, and label in the notebook output
display(m)
display(button)
display(coord_label)

Map(center=[40.08546, -4.08546], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

Button(description='Obtener Coordenadas', style=ButtonStyle())

Label(value='')